## Comparison of Prophet and TorchProphet
Both the Prophet and the NeuralProphet model share the concept of decomposing a time series into it's components which allows a human to inspect and interprete the individual components of the forecast. Since NeuralProphet differs both in the functionality and the interface, the TorchProphet allows users to rapidly explore NeuralProphet's capabilities. In the following we will walk through the differences and similarities when working with TorchProphet instead of Prophet.

### Identical behaviour
- **Trend**  
  Both Prophet and TorchProphet provide a trend component that can be predicted using `predict_trend(...)` and configured using the `changepoints` or `n_changepoints` argument during initialization.
- **Seasonality**  
  Both tools allow to model seasonality by using the `add_seasonality(...)` function or providing the `yearly_seasonality`, `weekly_seasonality`, `daily_seasonality` or `seasonality_mode` argument during initialization.
- **Country holidays**  
  Both tools allow to add country holidays as special events to the forecast. This is archieved by using the `add_country_holidays(...)` function.
- **Future regressors**  
  Regressors can be added using the `add_regressor(...)` function in both tools.
  > **Info:** In TorchProphet, we differentiate between lagged and future regressors. The `add_regressor(...)` function of Prophet corresponds to the `add_future_regressor(...)` function of TorchProphet.
- **Holidays**  
  In Prophet, additional events to the country holidays can be added during initialization by passing an `holidays` argument. In TorchProphet, this can be archieved by passing the holidays to the `add_events(...)` function. TorchProphet supports both methods.

### Different behaviour
- **Regularization**  
  In Prophet, the argument `prior_scale` can be used to configure regularization. In TorchProphet, regularization is controlled via the `regularization` argument. `prior_scale` and `regularization` have an inverse relationship but cannot directly be translated. Thus, TorchProphet does not support regularization via `prior_scale`. Regularization is archieved via the `_reg` argument.
  ```python
  # Prophet
  from prophet import Prophet
  m = Prophet(seasonality_prior_scale=0.5)
  # Wrapper
  from neuralprophet import TorchProphet as Prophet
  m = Prophet(seasonality_reg=0.5)
  ```
- **Uncertainty**  
  In Prophet, all forecasts are configured to use uncertainty intervals automatically. In NeuralProphet, the uncertaintly intervals need to be configured by the user. TorchProphet uses the default uncertainty intervals as defined in Prophet to mirror the behviour.

  However, the uncertainty interval calculation differs between Prophet and NeuralProphet. Since Prophet uses a MAP estimate for uncertainty by default [Thread](https://github.com/facebook/prophet/issues/1124), NeuralProphet relies on quantile regression. Thus, the values are not directly comparable.
  ```python
  # Prophet
  from prophet import Prophet
  m = Prophet(interval_width=0.8)
  # Wrapper
  from neuralprophet import TorchProphet as Prophet
  m = Prophet(interval_width=[0.1, 0.9])
  ```


### Unsupported features in TorchProphet
- **Saturating forecasts**  
  Saturating forecasts limit the predicted value to a certain limit called capacity. In Prophet, this is archieved by passing the `growth='logistic'` argument during initialization and providing a capacity limit. This functionality is currently not supported by NeuralProphet.
- **Conditional seasonality**  
  Conditional seasonality allows to model certain events as seasonal effects (eg. whether it's currently NFL season). This can be archieved in Prophet by passing the argument `condition_name` to `add_seasonality(...)`. This feature is currently also not supported in NeuralProphet.

### Additional features of TorchProphet
- **Autoregression**  
  TorchProphet allows to model autoregression of arbitrary lag lengths by building on a Neural Network implementation of the autoregression algorithm (called AR-Net). More information can be found here [Autoregression](https://neuralprophet.com/html/autoregression_yosemite_temps.html).
- **Lagged regressors**  
  TorchProphet does not only support "future" regressors like in Prophet, but also lagged regressors that are simultaneous to the time series to predict. More information can be found here [Lagged covariates](https://neuralprophet.com/html/lagged_covariates_energy_ercot.html).
- **Global model**  
  TorchProphet supports hierachical forecasts by training a global model on many simultaneous time series that is used to improve the performance of predicting an individual time series. More infos can be found here [Global Model](https://neuralprophet.com/html/global_modeling.html).
- **Neural Network architecture**  
  TorchProphet models the forecast components using a Neural Network. By default, the network has only one hidden layer. However, for more complex time series, the depth of the network can be increased to learn more complex relations.
- **PyTorch**  
  TorchProphet is build on Pytorch (soon PyTorch Lightning) and thus provides interfaces for developers to extend the vanilla model for specific use cases.

In [4]:
import logging
import warnings
import pandas as pd
from prophet import Prophet
from neuralprophet import NeuralProphet, set_log_level

set_log_level("ERROR")
logging.getLogger('prophet').setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')

### Trend

In [ ]:
# Prophet
p = Prophet(growth="linear", n_changepoints=10, changepoint_range=0.5)
p.fit(df)

# NeuralProphet
np = NeuralProphet(growth="linear", n_changepoints=10, changepoints_range=0.5)
np.fit(df)

### Seasonality

In [ ]:
# Prophet
p = Prophet(weekly_seasonality=False)
p = p.add_seasonality(name="monthly", period=30.5, fourier_order=5)
p.fit(df)

# NeuralProphet
np = NeuralProphet(weekly_seasonality=False)
np = np.add_seasonality(name="monthly", period=30.5, fourier_order=5)
np.fit(df)

### Country holidays

In [ ]:
# Prophet
p = Prophet()
p.add_country_holidays(country_name='US')
p.fit(df)

# NeuralProphet
np = NeuralProphet()
np.add_country_holidays(country_name='US')
np.fit(df)

### Future regressors

### Holidays & Events

### Regularization

### Uncertainty